# Importing Packages and creating universally useful tools

In [1]:
import numpy as np
import pandas as pd
import re
import requests
from collections import Counter
import bs4 as bs
import unicodedata
mdict = {"January":"1", "February":"2", "March":"3", "April":"4", "May":"5", "June":"6", "July":"7",\
         "August":"8", "September":"9", "October":"10", "November":"11", "December":"12"}
import us
STATES_DICT = us.states.mapping('name', 'abbr')

def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)
def stripper(begins):
    begins = [strip_accents(boo) for boo in begins]
    begins = [re.sub(r"%C3%BA","u", u) for u in begins]
    begins = [re.sub(r"%C3%A9","e", e) for e in begins]
    begins = [re.sub(r"%C3%AD","i", i) for i in begins]
    begins = [re.sub(r"%C3%A1","a", a) for a in begins]
    begins = [re.sub("%27", "'", nra) for nra in begins]
    #begins = [re.sub(" [A-Z]\. ", " ", nra) for nra in begins]
    return(begins)




## It begins

In [2]:
section = pd.read_html("https://en.wikipedia.org/wiki/List_of_current_United_States_governors")[0]
section = section.iloc[2:len(section),:]
section.columns = ["State", "Chamber", "Name", "DeleteA", "Party", "DeleteB", "Date", "End", "DeleteC"]
del(section["DeleteA"])
del(section["DeleteB"])
del(section["DeleteC"])
section["Chamber"] = ["governor" for x in range(len(section))]
section["State"] = section["State"].str.lower()
section["Party"] = section["Party"].str.lower()
section["Name"] = section["Name"].str.lower()
section["End"] = [re.findall("\d\d\d\d",x)[0] for x in section["End"]]
dates = section["Date"].tolist()
year = [re.findall("\d\d\d\d",x)[0] for x in dates]
day = [re.sub(",","",re.findall("\d+,",x)[0]) for x in dates]
month = [mdict[re.findall("^[A-Z][a-z]+",x)[0]] for x in dates]
df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
date = pd.to_datetime(df)
section = section.reset_index(drop = True)
section["Date"] = date
section["Party"] = [x[0] for x in section["Party"]]
section = section.sort_values(by = "Date").reset_index(drop=True)
section

,State,Chamber,Name,Party,Date,End
0,idaho,governor,butch otter,r,2007-01-01,2019
1,utah,governor,gary herbert,r,2009-08-11,2021
2,new york,governor,andrew cuomo,d,2011-01-01,2018
3,new mexico,governor,susana martinez,r,2011-01-01,2019
4,michigan,governor,rick snyder,r,2011-01-01,2019
5,wisconsin,governor,scott walker,r,2011-01-03,2019
6,nevada,governor,brian sandoval,r,2011-01-03,2019
7,california,governor,jerry brown,d,2011-01-03,2019
8,minnesota,governor,mark dayton,d,2011-01-03,2019
9,wyoming,governor,matt mead,r,2011-01-03,2019


In [3]:
len(section["Date"].unique())-2

35

In [4]:
#1
d20180601 = "https://web.archive.org/web/20180401205759/https://en.wikipedia.org/wiki/List_of_current_United_States_governors" 
#2
d20180131 = "https://web.archive.org/web/20180116222023/https://en.wikipedia.org/wiki/List_of_current_United_States_governors"
#3
d20180116 = "https://web.archive.org/web/20180114033616/https://en.wikipedia.org/wiki/List_of_current_United_States_governors"
#4
d20180113 = "https://web.archive.org/web/20171230015037/https://en.wikipedia.org/wiki/List_of_current_United_States_governors"
#5
d20170524 = "https://web.archive.org/web/20170523091423/https://en.wikipedia.org/wiki/List_of_current_United_States_governors"
#6
d20170410 = "https://web.archive.org/web/20170324035219/https://en.wikipedia.org/wiki/List_of_current_United_States_governors"
#7
d20170124 = "https://web.archive.org/web/20170121030531/https://en.wikipedia.org/wiki/List_of_current_United_States_governors"
#8
d20170117 = "https://web.archive.org/web/20170115201600/https://en.wikipedia.org/wiki/List_of_current_United_States_governors"
#9
#d20170116 = ""
#10
d20170109 = "https://web.archive.org/web/20170101061526/https://en.wikipedia.org/wiki/List_of_current_United_States_governors"
#11
#d20170105 = ""
#12
d20170101 = "https://web.archive.org/web/20161212222454/https://en.wikipedia.org/wiki/List_of_current_United_States_governors"
#13
#d20161215 = ""
#14
d20160111 = "https://web.archive.org/web/20160108225413/https://en.wikipedia.org/wiki/List_of_current_United_States_governors"
#15
d20151208 = "https://web.archive.org/web/20151128152354/https://en.wikipedia.org/wiki/List_of_current_United_States_governors"
#16
d20150218 = "https://web.archive.org/web/20150212061706/https://en.wikipedia.org/wiki/List_of_current_United_States_governors"
#17
d20150121 = "https://web.archive.org/web/20150109192911/https://en.wikipedia.org/wiki/List_of_current_United_States_governors"
#18
#d20150120 = ""
#19
#d20150113 = ""
#20
#d20150112 = ""
#21
d20150108 = "https://web.archive.org/web/20141229094208/https://en.wikipedia.org/wiki/List_of_current_United_States_governors"
#22
#d20150106 = ""
#23
#d20150105 = ""
#24
d20141201 = "https://web.archive.org/web/20141124114713/https://en.wikipedia.org/wiki/List_of_current_United_States_governors"
#25
d20130116 = "https://web.archive.org/web/20130109011251/https://en.wikipedia.org/wiki/List_of_current_United_States_governors"
#26
d20130107 = "https://web.archive.org/web/20121128004601/https://en.wikipedia.org/wiki/List_of_current_United_States_governors"
#27
d20120110 = "https://web.archive.org/web/20111022055830/https://en.wikipedia.org/wiki/List_of_current_United_States_governors"
#28
d20110115 = "https://web.archive.org/web/20110104210717/https://en.wikipedia.org/wiki/List_of_current_United_States_governors"
#29
#d20110111 = ""
#30
#d20110110 = ""
#31
#d20110108 = ""
#32
#d20110105 = ""
#33
d20110104 = "https://web.archive.org/web/20101128052602/https://en.wikipedia.org/wiki/List_of_current_United_States_governors"
#34
#d20110103 = ""
#35
#d20110101 = ""


In [5]:
archive = [d20180601, d20180131,\
           d20180116, d20180113,\
           d20170524, d20170410,\
           d20170124, d20170117,\
           d20170109, d20170101,\
           d20160111, d20151208,\
           d20150218, d20150121,\
           d20150108, d20141201,\
           d20130116, d20130107,\
           d20120110, d20110115,\
           d20110104]

In [6]:
len(archive)

21

In [7]:
section = pd.read_html(archive[0])[2]
section = section.iloc[2:len(section),:]
section.columns = ["State", "Chamber", "Name", "DeleteA", "Party", "DeleteB", "Date", "End", "DeleteC"]
del(section["DeleteA"])
del(section["DeleteB"])
del(section["DeleteC"])
section["Chamber"] = ["governor" for x in range(len(section))]
section["State"] = section["State"].str.lower()
section["Party"] = section["Party"].str.lower()
section["Name"] = section["Name"].str.lower()
section["End"] = [re.findall("\d\d\d\d",x)[0] for x in section["End"]]
dates = section["Date"].tolist()
year = [re.findall("\d\d\d\d",x)[0] for x in dates]
day = [re.sub(",","",re.findall("\d+,",x)[0]) for x in dates]
month = [mdict[re.findall("^[A-Z][a-z]+",x)[0]] for x in dates]
df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
date = pd.to_datetime(df)
section = section.reset_index(drop = True)
section["Date"] = date
section["Party"] = [x[0] for x in section["Party"]]
section = section.sort_values(by = "Date").reset_index(drop=True)
weird = section["Name"].str.replace(",","")
section["Name"] = pd.Series([x[int(len(x)/2):len(x)] for x in weird])
section0 = section.copy(deep=True)
section0

,State,Chamber,Name,Party,Date,End
0,idaho,governor,butch otter,r,2007-01-01,2019
1,utah,governor,gary herbert,r,2009-08-11,2021
2,new york,governor,andrew cuomo,d,2011-01-01,2018
3,new mexico,governor,susana martinez,r,2011-01-01,2019
4,michigan,governor,rick snyder,r,2011-01-01,2019
5,wyoming,governor,matt mead,r,2011-01-03,2019
6,nevada,governor,brian sandoval,r,2011-01-03,2019
7,california,governor,jerry brown,d,2011-01-03,2019
8,wisconsin,governor,scott walker,r,2011-01-03,2019
9,minnesota,governor,mark dayton,d,2011-01-03,2019


In [8]:
section = pd.read_html(archive[1])[3]
section = section.iloc[2:len(section),:]
section.columns = ["State", "Chamber", "Name", "DeleteA", "Party", "DeleteB", "Date", "End", "DeleteC"]
del(section["DeleteA"])
del(section["DeleteB"])
del(section["DeleteC"])
section["Chamber"] = ["governor" for x in range(len(section))]
section["State"] = section["State"].str.lower()
section["Party"] = section["Party"].str.lower()
section["Name"] = section["Name"].str.lower()
section["End"] = [re.findall("\d\d\d\d",x)[0] for x in section["End"]]
dates = section["Date"].tolist()
year = [re.findall("\d\d\d\d",x)[0] for x in dates]
day = [re.sub(",","",re.findall("\d+,",x)[0]) for x in dates]
month = [mdict[re.findall("^[A-Z][a-z]+",x)[0]] for x in dates]
df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
date = pd.to_datetime(df)
section = section.reset_index(drop = True)
section["Date"] = date
section["Party"] = [x[0] for x in section["Party"]]
section = section.sort_values(by = "Date").reset_index(drop=True)
weird = section["Name"].str.replace(",","")
section["Name"] = pd.Series([x[int(len(x)/2):len(x)] for x in weird])
section1 = section.copy(deep=True)
section1

,State,Chamber,Name,Party,Date,End
0,idaho,governor,butch otter,r,2007-01-01,2019
1,utah,governor,gary herbert,r,2009-08-11,2021
2,new york,governor,andrew cuomo,d,2011-01-01,2018
3,new mexico,governor,susana martinez,r,2011-01-01,2019
4,michigan,governor,rick snyder,r,2011-01-01,2019
5,wyoming,governor,matt mead,r,2011-01-03,2019
6,nevada,governor,brian sandoval,r,2011-01-03,2019
7,california,governor,jerry brown,d,2011-01-03,2019
8,wisconsin,governor,scott walker,r,2011-01-03,2019
9,minnesota,governor,mark dayton,d,2011-01-03,2019


In [9]:
section = pd.read_html(archive[2])[3]
section = section.iloc[2:len(section),:]
section.columns = ["State", "Chamber", "Name", "DeleteA", "Party", "DeleteB", "Date", "End", "DeleteC"]
del(section["DeleteA"])
del(section["DeleteB"])
del(section["DeleteC"])
section["Chamber"] = ["governor" for x in range(len(section))]
section["State"] = section["State"].str.lower()
section["Party"] = section["Party"].str.lower()
section["Name"] = section["Name"].str.lower()
section["End"] = [re.findall("\d\d\d\d",x)[0] for x in section["End"]]
dates = section["Date"].tolist()
year = [re.findall("\d\d\d\d",x)[0] for x in dates]
day = [re.sub(",","",re.findall("\d+,",x)[0]) for x in dates]
month = [mdict[re.findall("^[A-Z][a-z]+",x)[0]] for x in dates]
df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
date = pd.to_datetime(df)
section = section.reset_index(drop = True)
section["Date"] = date
section["Party"] = [x[0] for x in section["Party"]]
section = section.sort_values(by = "Date").reset_index(drop=True)
weird = section["Name"].str.replace(",","")
section["Name"] = pd.Series([x[int(len(x)/2):len(x)] for x in weird])
section2 = section.copy(deep=True)
section2

,State,Chamber,Name,Party,Date,End
0,idaho,governor,butch otter,r,2007-01-01,2019
1,utah,governor,gary herbert,r,2009-08-11,2021
2,new jersey,governor,chris christie,r,2010-01-19,2018
3,michigan,governor,rick snyder,r,2011-01-01,2019
4,new york,governor,andrew cuomo,d,2011-01-01,2018
5,new mexico,governor,susana martinez,r,2011-01-01,2019
6,wyoming,governor,matt mead,r,2011-01-03,2019
7,minnesota,governor,mark dayton,d,2011-01-03,2019
8,california,governor,jerry brown,d,2011-01-03,2019
9,nevada,governor,brian sandoval,r,2011-01-03,2019


In [10]:
section = pd.read_html(archive[3])[3]
section = section.iloc[2:len(section),:]
section.columns = ["State", "Chamber", "Name", "DeleteA", "Party", "DeleteB", "Date", "End", "DeleteC"]
del(section["DeleteA"])
del(section["DeleteB"])
del(section["DeleteC"])
section["Chamber"] = ["governor" for x in range(len(section))]
section["State"] = section["State"].str.lower()
section["Party"] = section["Party"].str.lower()
section["Name"] = section["Name"].str.lower()
section["End"] = [re.findall("\d\d\d\d",x)[0] for x in section["End"]]
dates = section["Date"].tolist()
year = [re.findall("\d\d\d\d",x)[0] for x in dates]
day = [re.sub(",","",re.findall("\d+,",x)[0]) for x in dates]
month = [mdict[re.findall("^[A-Z][a-z]+",x)[0]] for x in dates]
df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
date = pd.to_datetime(df)
section = section.reset_index(drop = True)
section["Date"] = date
section["Party"] = [x[0] for x in section["Party"]]
section = section.sort_values(by = "Date").reset_index(drop=True)
weird = section["Name"].str.replace(",","")
section["Name"] = pd.Series([x[int(len(x)/2):len(x)] for x in weird])
section3 = section.copy(deep=True)
section3

,State,Chamber,Name,Party,Date,End
0,idaho,governor,butch otter,r,2007-01-01,2019
1,utah,governor,gary herbert,r,2009-08-11,2021
2,new jersey,governor,chris christie,r,2010-01-19,2018
3,michigan,governor,rick snyder,r,2011-01-01,2019
4,new york,governor,andrew cuomo,d,2011-01-01,2018
5,new mexico,governor,susana martinez,r,2011-01-01,2019
6,nevada,governor,brian sandoval,r,2011-01-03,2019
7,wisconsin,governor,scott walker,r,2011-01-03,2019
8,minnesota,governor,mark dayton,d,2011-01-03,2019
9,wyoming,governor,matt mead,r,2011-01-03,2019


In [11]:
section = pd.read_html(archive[4])[3]
section = section.iloc[2:len(section),:]
section.columns = ["State", "Chamber", "Name", "DeleteA", "Party", "DeleteB", "Date", "End", "DeleteC"]
del(section["DeleteA"])
del(section["DeleteB"])
del(section["DeleteC"])
section["Chamber"] = ["governor" for x in range(len(section))]
section["State"] = section["State"].str.lower()
section["Party"] = section["Party"].str.lower()
section["Name"] = section["Name"].str.lower()
section["End"] = [re.findall("\d\d\d\d",x)[0] for x in section["End"]]
dates = section["Date"].tolist()
year = [re.findall("\d\d\d\d",x)[0] for x in dates]
day = [re.sub(",","",re.findall("\d+,",x)[0]) for x in dates]
month = [mdict[re.findall("^[A-Z][a-z]+",x)[0]] for x in dates]
df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
date = pd.to_datetime(df)
section = section.reset_index(drop = True)
section["Date"] = date
section["Party"] = [x[0] for x in section["Party"]]
section = section.sort_values(by = "Date").reset_index(drop=True)
weird = section["Name"].str.replace(",","")
section["Name"] = pd.Series([x[int(len(x)/2):len(x)] for x in weird])
section4 = section.copy(deep=True)
section4

,State,Chamber,Name,Party,Date,End
0,idaho,governor,butch otter,r,2007-01-01,2019
1,utah,governor,gary herbert,r,2009-08-11,2021
2,new jersey,governor,chris christie,r,2010-01-19,2018
3,new york,governor,andrew cuomo,d,2011-01-01,2018
4,new mexico,governor,susana martinez,r,2011-01-01,2019
5,michigan,governor,rick snyder,r,2011-01-01,2019
6,wisconsin,governor,scott walker,r,2011-01-03,2019
7,nevada,governor,brian sandoval,r,2011-01-03,2019
8,minnesota,governor,mark dayton,d,2011-01-03,2019
9,california,governor,jerry brown,d,2011-01-03,2019


In [12]:
section = pd.read_html(archive[5])[3]
section = section.iloc[2:len(section),:]
section.columns = ["State", "Chamber", "Name", "DeleteA", "Party", "DeleteB", "Date", "End", "DeleteC"]
del(section["DeleteA"])
del(section["DeleteB"])
del(section["DeleteC"])
section["Chamber"] = ["governor" for x in range(len(section))]
section["State"] = section["State"].str.lower()
section["Party"] = section["Party"].str.lower()
section["Name"] = section["Name"].str.lower()
section["End"] = [re.findall("\d\d\d\d",x)[0] for x in section["End"]]
dates = section["Date"].tolist()
year = [re.findall("\d\d\d\d",x)[0] for x in dates]
day = [re.sub(",","",re.findall("\d+,",x)[0]) for x in dates]
month = [mdict[re.findall("^[A-Z][a-z]+",x)[0]] for x in dates]
df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
date = pd.to_datetime(df)
section = section.reset_index(drop = True)
section["Date"] = date
section["Party"] = [x[0] for x in section["Party"]]
section = section.sort_values(by = "Date").reset_index(drop=True)
weird = section["Name"].str.replace(",","")
section["Name"] = pd.Series([x[int(len(x)/2):len(x)] for x in weird])
section5 = section.copy(deep=True)
section5

,State,Chamber,Name,Party,Date,End
0,idaho,governor,butch otter,r,2007-01-01,2019
1,utah,governor,gary herbert,r,2009-08-11,2021
2,new jersey,governor,chris christie,r,2010-01-19,2018
3,michigan,governor,rick snyder,r,2011-01-01,2019
4,new mexico,governor,susana martinez,r,2011-01-01,2019
5,new york,governor,andrew cuomo,d,2011-01-01,2018
6,wisconsin,governor,scott walker,r,2011-01-03,2019
7,nevada,governor,brian sandoval,r,2011-01-03,2019
8,minnesota,governor,mark dayton,d,2011-01-03,2019
9,wyoming,governor,matt mead,r,2011-01-03,2019


In [13]:
section = pd.read_html(archive[6])[3]
section = section.iloc[2:len(section),:]
section.columns = ["State", "Chamber", "Name", "DeleteA", "Party", "DeleteB", "Date", "End", "DeleteC"]
del(section["DeleteA"])
del(section["DeleteB"])
del(section["DeleteC"])
section["Chamber"] = ["governor" for x in range(len(section))]
section["State"] = section["State"].str.lower()
section["Party"] = section["Party"].str.lower()
section["Name"] = section["Name"].str.lower()
section["End"] = [re.findall("\d\d\d\d",x)[0] for x in section["End"]]
dates = section["Date"].tolist()
year = [re.findall("\d\d\d\d",x)[0] for x in dates]
day = [re.sub(",","",re.findall("\d+,",x)[0]) for x in dates]
month = [mdict[re.findall("^[A-Z][a-z]+",x)[0]] for x in dates]
df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
date = pd.to_datetime(df)
section = section.reset_index(drop = True)
section["Date"] = date
section["Party"] = [x[0] for x in section["Party"]]
section = section.sort_values(by = "Date").reset_index(drop=True)
weird = section["Name"].str.replace(",","")
section["Name"] = pd.Series([x[int(len(x)/2):len(x)] for x in weird])
section6 = section.copy(deep=True)
section6

,State,Chamber,Name,Party,Date,End
0,idaho,governor,butch otter,r,2007-01-01,2019
1,utah,governor,gary herbert,r,2009-08-11,2021
2,new jersey,governor,chris christie,r,2010-01-19,2018
3,new mexico,governor,susana martinez,r,2011-01-01,2019
4,new york,governor,andrew cuomo,d,2011-01-01,2018
5,michigan,governor,rick snyder,r,2011-01-01,2019
6,wisconsin,governor,scott walker,r,2011-01-03,2019
7,nevada,governor,brian sandoval,r,2011-01-03,2019
8,wyoming,governor,matt mead,r,2011-01-03,2019
9,california,governor,jerry brown,d,2011-01-03,2019


In [14]:
section = pd.read_html(archive[7])[3]
section = section.iloc[2:len(section),:]
section.columns = ["State", "Chamber", "Name", "DeleteA", "Party", "DeleteB", "Date", "End", "DeleteC"]
del(section["DeleteA"])
del(section["DeleteB"])
del(section["DeleteC"])
section["Chamber"] = ["governor" for x in range(len(section))]
section["State"] = section["State"].str.lower()
section["Party"] = section["Party"].str.lower()
section["Name"] = section["Name"].str.lower()
section["End"] = [re.findall("\d\d\d\d",x)[0] for x in section["End"]]
dates = section["Date"].tolist()
year = [re.findall("\d\d\d\d",x)[0] for x in dates]
day = [re.sub(",","",re.findall("\d+,",x)[0]) for x in dates]
month = [mdict[re.findall("^[A-Z][a-z]+",x)[0]] for x in dates]
df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
date = pd.to_datetime(df)
section = section.reset_index(drop = True)
section["Date"] = date
section["Party"] = [x[0] for x in section["Party"]]
section = section.sort_values(by = "Date").reset_index(drop=True)
weird = section["Name"].str.replace(",","")
section["Name"] = pd.Series([x[int(len(x)/2):len(x)] for x in weird])
section7 = section.copy(deep=True)
section7

,State,Chamber,Name,Party,Date,End
0,idaho,governor,butch otter,r,2007-01-01,2019
1,delaware,governor,jack markell,d,2009-01-20,2017
2,utah,governor,gary herbert,r,2009-08-11,2021
3,new jersey,governor,chris christie,r,2010-01-19,2018
4,michigan,governor,rick snyder,r,2011-01-01,2019
5,new mexico,governor,susana martinez,r,2011-01-01,2019
6,new york,governor,andrew cuomo,d,2011-01-01,2018
7,nevada,governor,brian sandoval,r,2011-01-03,2019
8,minnesota,governor,mark dayton,d,2011-01-03,2019
9,wyoming,governor,matt mead,r,2011-01-03,2019


In [15]:
section = pd.read_html(archive[8])[3]
section = section.iloc[2:len(section),:]
section.columns = ["State", "Chamber", "Name", "DeleteA", "Party", "DeleteB", "Date", "End", "DeleteC"]
del(section["DeleteA"])
del(section["DeleteB"])
del(section["DeleteC"])
section["Chamber"] = ["governor" for x in range(len(section))]
section["State"] = section["State"].str.lower()
section["Party"] = section["Party"].str.lower()
section["Name"] = section["Name"].str.lower()
section["End"] = [re.findall("\d\d\d\d",x)[0] for x in section["End"]]
dates = section["Date"].tolist()
year = [re.findall("\d\d\d\d",x)[0] for x in dates]
day = [re.sub(",","",re.findall("\d+,",x)[0]) for x in dates]
month = [mdict[re.findall("^[A-Z][a-z]+",x)[0]] for x in dates]
df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
date = pd.to_datetime(df)
section = section.reset_index(drop = True)
section["Date"] = date
section["Party"] = [x[0] for x in section["Party"]]
section = section.sort_values(by = "Date").reset_index(drop=True)
weird = section["Name"].str.replace(",","")
section["Name"] = pd.Series([x[int(len(x)/2):len(x)] for x in weird])
section8 = section.copy(deep=True)
section8

,State,Chamber,Name,Party,Date,End
0,idaho,governor,butch otter,r,2007-01-01,2019
1,missouri,governor,jay nixon,d,2009-01-12,2017
2,delaware,governor,jack markell,d,2009-01-20,2017
3,utah,governor,gary herbert,r,2009-08-11,2021
4,new jersey,governor,chris christie,r,2010-01-19,2018
5,new york,governor,andrew cuomo,d,2011-01-01,2018
6,new mexico,governor,susana martinez,r,2011-01-01,2019
7,michigan,governor,rick snyder,r,2011-01-01,2019
8,nevada,governor,brian sandoval,r,2011-01-03,2019
9,wisconsin,governor,scott walker,r,2011-01-03,2019


In [16]:
section = pd.read_html(archive[9])[3]
section = section.iloc[2:len(section),:]
section.columns = ["State", "Chamber", "Name", "DeleteA", "Party", "DeleteB", "Date", "End", "DeleteC"]
del(section["DeleteA"])
del(section["DeleteB"])
del(section["DeleteC"])
section["Chamber"] = ["governor" for x in range(len(section))]
section["State"] = section["State"].str.lower()
section["Party"] = section["Party"].str.lower()
section["Name"] = section["Name"].str.lower()
section["End"] = [re.findall("\d\d\d\d",x)[0] for x in section["End"]]
dates = section["Date"].tolist()
year = [re.findall("\d\d\d\d",x)[0] for x in dates]
day = [re.sub(",","",re.findall("\d+,",x)[0]) for x in dates]
month = [mdict[re.findall("^[A-Z][a-z]+",x)[0]] for x in dates]
df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
date = pd.to_datetime(df)
section = section.reset_index(drop = True)
section["Date"] = date
section["Party"] = [x[0] for x in section["Party"]]
section = section.sort_values(by = "Date").reset_index(drop=True)
weird = section["Name"].str.replace(",","")
section["Name"] = pd.Series([x[int(len(x)/2):len(x)] for x in weird])
section9 = section.copy(deep=True)
section9

,State,Chamber,Name,Party,Date,End
0,idaho,governor,butch otter,r,2007-01-01,2019
1,missouri,governor,jay nixon,d,2009-01-12,2017
2,delaware,governor,jack markell,d,2009-01-20,2017
3,utah,governor,gary herbert,r,2009-08-11,2021
4,new jersey,governor,chris christie,r,2010-01-19,2018
5,north dakota,governor,jack dalrymple,r,2010-12-07,2016
6,new york,governor,andrew cuomo,d,2011-01-01,2018
7,new mexico,governor,susana martinez,r,2011-01-01,2019
8,michigan,governor,rick snyder,r,2011-01-01,2019
9,nevada,governor,brian sandoval,r,2011-01-03,2019


In [17]:
section = pd.read_html(archive[10])[3]
section = section.iloc[1:len(section),:]
section.columns = ["State", "Chamber", "Name", "Party", "DeleteB", "Date", "End", "DeleteC"]
del(section["DeleteB"])
del(section["DeleteC"])
section["Chamber"] = ["governor" for x in range(len(section))]
section["State"] = section["State"].str.lower()
section["Party"] = section["Party"].str.lower()
section["Name"] = section["Name"].str.lower()
section["End"] = [re.findall("\d\d\d\d",x)[0] for x in section["End"]]
dates = section["Date"].tolist()
year = [re.findall("\d\d\d\d",x)[0] for x in dates]
day = [re.sub(",","",re.findall("\d+,",x)[0]) for x in dates]
month = [mdict[re.findall("^[A-Z][a-z]+",x)[0]] for x in dates]
df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
date = pd.to_datetime(df)
section = section.reset_index(drop = True)
section["Date"] = date
section["Party"] = [x[0] for x in section["Party"]]
section = section.sort_values(by = "Date").reset_index(drop=True)
weird = section["Name"].str.replace(",","")
section["Name"] = pd.Series([x[int(len(x)/2):len(x)] for x in weird])
section10 = section.copy(deep=True)
section10

,State,Chamber,Name,Party,Date,End
0,idaho,governor,butch otter,r,2007-01-01,2019
1,louisiana,governor,bobby jindal,r,2008-01-14,2016
2,missouri,governor,jay nixon,d,2009-01-12,2017
3,delaware,governor,jack markell,d,2009-01-20,2017
4,utah,governor,gary herbert,r,2009-08-11,2017
5,new jersey,governor,chris christie,r,2010-01-19,2018
6,west virginia,governor,earl ray tomblin,d,2010-11-15,2017
7,north dakota,governor,jack dalrymple,r,2010-12-07,2016
8,new york,governor,andrew cuomo,d,2011-01-01,2019
9,new mexico,governor,susana martinez,r,2011-01-01,2019


In [18]:
section = pd.read_html(archive[11])[3]
section = section.iloc[1:len(section),:]
section.columns = ["State", "Chamber", "Name", "Party", "DeleteB", "Date", "End", "DeleteC"]
del(section["DeleteB"])
del(section["DeleteC"])
section["Chamber"] = ["governor" for x in range(len(section))]
section["State"] = section["State"].str.lower()
section["Party"] = section["Party"].str.lower()
section["Name"] = section["Name"].str.lower()
section["End"] = [re.findall("\d\d\d\d",x)[0] for x in section["End"]]
dates = section["Date"].tolist()
year = [re.findall("\d\d\d\d",x)[0] for x in dates]
day = [re.sub(",","",re.findall("\d+,",x)[0]) for x in dates]
month = [mdict[re.findall("^[A-Z][a-z]+",x)[0]] for x in dates]
df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
date = pd.to_datetime(df)
section = section.reset_index(drop = True)
section["Date"] = date
section["Party"] = [x[0] for x in section["Party"]]
section = section.sort_values(by = "Date").reset_index(drop=True)
weird = section["Name"].str.replace(",","")
section["Name"] = pd.Series([x[int(len(x)/2):len(x)] for x in weird])
section11 = section.copy(deep=True)
section11

,State,Chamber,Name,Party,Date,End
0,idaho,governor,butch otter,r,2007-01-01,2019
1,kentucky,governor,steve beshear,d,2007-12-11,2015
2,louisiana,governor,bobby jindal,r,2008-01-14,2016
3,missouri,governor,jay nixon,d,2009-01-12,2017
4,delaware,governor,jack markell,d,2009-01-20,2017
5,utah,governor,gary herbert,r,2009-08-11,2017
6,new jersey,governor,chris christie,r,2010-01-19,2018
7,west virginia,governor,earl ray tomblin,d,2010-11-15,2017
8,north dakota,governor,jack dalrymple,r,2010-12-07,2016
9,new york,governor,andrew cuomo,d,2011-01-01,2019


In [19]:
section = pd.read_html(archive[12])[3]
section = section.iloc[1:len(section),:]
section.columns = ["State", "Chamber", "Name", "Party", "DeleteB", "Date", "End", "DeleteC"]
del(section["DeleteB"])
del(section["DeleteC"])
section["Chamber"] = ["governor" for x in range(len(section))]
section["State"] = section["State"].str.lower()
section["Party"] = section["Party"].str.lower()
section["Name"] = section["Name"].str.lower()
section["End"] = [re.findall("\d\d\d\d",x)[0] for x in section["End"]]
dates = section["Date"].tolist()
year = [re.findall("\d\d\d\d",x)[0] for x in dates]
day = [re.sub(",","",re.findall("\d+,",x)[0]) for x in dates]
month = [mdict[re.findall("^[A-Z][a-z]+",x)[0]] for x in dates]
df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
date = pd.to_datetime(df)
section = section.reset_index(drop = True)
section["Date"] = date
section["Party"] = [x[0] for x in section["Party"]]
section = section.sort_values(by = "Date").reset_index(drop=True)
weird = section["Name"].str.replace(",","")
section["Name"] = pd.Series([x[int(len(x)/2):len(x)] for x in weird])
section12 = section.copy(deep=True)
section12

,State,Chamber,Name,Party,Date,End
0,idaho,governor,butch otter,r,2007-01-01,2019
1,kentucky,governor,steve beshear,d,2007-12-11,2015
2,louisiana,governor,bobby jindal,r,2008-01-14,2016
3,missouri,governor,jay nixon,d,2009-01-12,2017
4,delaware,governor,jack markell,d,2009-01-20,2017
5,utah,governor,gary herbert,r,2009-08-11,2017
6,new jersey,governor,chris christie,r,2010-01-19,2018
7,west virginia,governor,earl ray tomblin,d,2010-11-15,2017
8,north dakota,governor,jack dalrymple,r,2010-12-07,2016
9,new york,governor,andrew cuomo,d,2011-01-01,2019


In [20]:
section = pd.read_html(archive[13])[3]
section = section.iloc[1:len(section),:]
section.columns = ["State", "Chamber", "Name", "Party", "DeleteB", "Date", "End", "DeleteC"]
del(section["DeleteB"])
del(section["DeleteC"])
section["Chamber"] = ["governor" for x in range(len(section))]
section["State"] = section["State"].str.lower()
section["Party"] = section["Party"].str.lower()
section["Name"] = section["Name"].str.lower()
section["End"] = [re.findall("\d\d\d\d",x)[0] for x in section["End"]]
dates = section["Date"].tolist()
year = [re.findall("\d\d\d\d",x)[0] for x in dates]
day = [re.sub(",","",re.findall("\d+,",x)[0]) for x in dates]
month = [mdict[re.findall("^[A-Z][a-z]+",x)[0]] for x in dates]
df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
date = pd.to_datetime(df)
section = section.reset_index(drop = True)
section["Date"] = date
section["Party"] = [x[0] for x in section["Party"]]
section = section.sort_values(by = "Date").reset_index(drop=True)
weird = section["Name"].str.replace(",","")
section["Name"] = pd.Series([x[int(len(x)/2):len(x)] for x in weird])
section13 = section.copy(deep=True)
section13

,State,Chamber,Name,Party,Date,End
0,texas,governor,rick perry,r,2000-12-21,2015
1,idaho,governor,butch otter,r,2007-01-01,2019
2,arkansas,governor,mike beebe,d,2007-01-09,2015
3,maryland,governor,martin o'malley,d,2007-01-17,2015
4,kentucky,governor,steve beshear,d,2007-12-11,2015
5,louisiana,governor,bobby jindal,r,2008-01-14,2016
6,missouri,governor,jay nixon,d,2009-01-12,2017
7,delaware,governor,jack markell,d,2009-01-20,2017
8,illinois,governor,pat quinn,d,2009-01-29,2015
9,utah,governor,gary herbert,r,2009-08-11,2017


In [21]:
section = pd.read_html(archive[14])[3]
section = section.iloc[1:len(section),:]
section.columns = ["State", "Chamber", "Name", "Party", "DeleteB", "Date", "End", "DeleteC"]
del(section["DeleteB"])
del(section["DeleteC"])
section["Chamber"] = ["governor" for x in range(len(section))]
section["State"] = section["State"].str.lower()
section["Party"] = section["Party"].str.lower()
section["Name"] = section["Name"].str.lower()
section["End"] = [re.findall("\d\d\d\d",x)[0] for x in section["End"]]
dates = section["Date"].tolist()
year = [re.findall("\d\d\d\d",x)[0] for x in dates]
day = [re.sub(",","",re.findall("\d+,",x)[0]) for x in dates]
month = [mdict[re.findall("^[A-Z][a-z]+",x)[0]] for x in dates]
df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
date = pd.to_datetime(df)
section = section.reset_index(drop = True)
section["Date"] = date
section["Party"] = [x[0] for x in section["Party"]]
section = section.sort_values(by = "Date").reset_index(drop=True)
weird = section["Name"].str.replace(",","")
section["Name"] = pd.Series([x[int(len(x)/2):len(x)] for x in weird])
section14 = section.copy(deep=True)
section14

,State,Chamber,Name,Party,Date,End
0,texas,governor,rick perry,r,2000-12-21,2015
1,nebraska,governor,dave heineman,r,2005-01-20,2015
2,idaho,governor,butch otter,r,2007-01-01,2015
3,massachusetts,governor,deval patrick,d,2007-01-04,2015
4,arkansas,governor,mike beebe,d,2007-01-09,2015
5,maryland,governor,martin o'malley,d,2007-01-17,2015
6,kentucky,governor,steve beshear,d,2007-12-11,2015
7,louisiana,governor,bobby jindal,r,2008-01-14,2016
8,missouri,governor,jay nixon,d,2009-01-12,2017
9,delaware,governor,jack markell,d,2009-01-20,2017


In [22]:
section = pd.read_html(archive[15])[3]
section = section.iloc[1:len(section),:]
section.columns = ["State", "Chamber", "Name", "Party", "DeleteB", "Date", "End", "DeleteC"]
del(section["DeleteB"])
del(section["DeleteC"])
section["Chamber"] = ["governor" for x in range(len(section))]
section["State"] = section["State"].str.lower()
section["Party"] = section["Party"].str.lower()
section["Name"] = section["Name"].str.lower()
section["End"] = [re.findall("\d\d\d\d",x)[0] for x in section["End"]]
dates = section["Date"].tolist()
year = [re.findall("\d\d\d\d",x)[0] for x in dates]
day = [re.sub(",","",re.findall("\d+,",x)[0]) for x in dates]
month = [mdict[re.findall("^[A-Z][a-z]+",x)[0]] for x in dates]
df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
date = pd.to_datetime(df)
section = section.reset_index(drop = True)
section["Date"] = date
section["Party"] = [x[0] for x in section["Party"]]
section = section.sort_values(by = "Date").reset_index(drop=True)
weird = section["Name"].str.replace(",","")
section["Name"] = pd.Series([x[int(len(x)/2):len(x)] for x in weird])
section15 = section.copy(deep=True)
section15

,State,Chamber,Name,Party,Date,End
0,texas,governor,rick perry,r,2000-12-21,2015
1,nebraska,governor,dave heineman,r,2005-01-20,2015
2,idaho,governor,butch otter,r,2007-01-01,2015
3,massachusetts,governor,deval patrick,d,2007-01-04,2015
4,arkansas,governor,mike beebe,d,2007-01-09,2015
5,maryland,governor,martin o'malley,d,2007-01-17,2015
6,kentucky,governor,steve beshear,d,2007-12-11,2015
7,louisiana,governor,bobby jindal,r,2008-01-14,2016
8,missouri,governor,jay nixon,d,2009-01-12,2017
9,delaware,governor,jack markell,d,2009-01-20,2017


In [23]:
section = pd.read_html(archive[16])[4]
section = section.iloc[1:len(section),:]
section.columns = ["State", "Chamber", "Name", "Party", "Date", "End", "DeleteC"]
del(section["DeleteC"])
section["Chamber"] = ["governor" for x in range(len(section))]
section["State"] = section["State"].str.lower()
section["Party"] = section["Party"].str.lower()
section["Name"] = section["Name"].str.lower()
section["End"] = [re.findall("\d\d\d\d",x)[0] for x in section["End"]]
dates = section["Date"].tolist()
year = [re.findall("\d\d\d\d",x)[0] for x in dates]
day = [re.sub(",","",re.findall("\d+,",x)[0]) for x in dates]
month = [mdict[re.findall("^[A-Z][a-z]+",x)[0]] for x in dates]
df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
date = pd.to_datetime(df)
section = section.reset_index(drop = True)
section["Date"] = date
section["Party"] = [x[0] for x in section["Party"]]
section = section.sort_values(by = "Date").reset_index(drop=True)
weird = section["Name"].str.replace(",","")
#section["Name"] = pd.Series([x[int(len(x)/2):len(x)] for x in weird])
section16 = section.copy(deep=True)
section16

,State,Chamber,Name,Party,Date,End
0,texas,governor,rick perry,r,2000-12-21,2015
1,indiana,governor,mitch daniels,r,2005-01-10,2013
2,washington,governor,christine gregoire,d,2005-01-12,2013
3,nebraska,governor,dave heineman,r,2005-01-20,2015
4,idaho,governor,butch otter,r,2007-01-01,2015
5,massachusetts,governor,deval patrick,d,2007-01-04,2015
6,arkansas,governor,mike beebe,d,2007-01-09,2015
7,maryland,governor,martin o'malley,d,2007-01-17,2015
8,kentucky,governor,steve beshear,d,2007-12-11,2015
9,louisiana,governor,bobby jindal,r,2008-01-14,2016


In [24]:
section = pd.read_html(archive[17])[4]
section = section.iloc[1:len(section),:]
section.columns = ["State", "Chamber", "Name", "Party", "Date", "End", "DeleteC"]
del(section["DeleteC"])
section["Chamber"] = ["governor" for x in range(len(section))]
section["State"] = section["State"].str.lower()
section["Party"] = section["Party"].str.lower()
section["Name"] = section["Name"].str.lower()
section["End"] = [re.findall("\d\d\d\d",x)[0] for x in section["End"]]
dates = section["Date"].tolist()
year = [re.findall("\d\d\d\d",x)[0] for x in dates]
day = [re.sub(",","",re.findall("\d+,",x)[0]) for x in dates]
month = [mdict[re.findall("^[A-Z][a-z]+",x)[0]] for x in dates]
df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
date = pd.to_datetime(df)
section = section.reset_index(drop = True)
section["Date"] = date
section["Party"] = [x[0] for x in section["Party"]]
section = section.sort_values(by = "Date").reset_index(drop=True)
weird = section["Name"].str.replace(",","")
#section["Name"] = pd.Series([x[int(len(x)/2):len(x)] for x in weird])
section17 = section.copy(deep=True)
section17

,State,Chamber,Name,Party,Date,End
0,texas,governor,rick perry,r,2000-12-21,2015
1,montana,governor,brian schweitzer,d,2005-01-03,2013
2,new hampshire,governor,john lynch,d,2005-01-06,2013
3,indiana,governor,mitch daniels,r,2005-01-10,2013
4,washington,governor,christine gregoire,d,2005-01-12,2013
5,nebraska,governor,dave heineman,r,2005-01-20,2015
6,idaho,governor,butch otter,r,2007-01-01,2015
7,massachusetts,governor,deval patrick,d,2007-01-04,2015
8,arkansas,governor,mike beebe,d,2007-01-09,2015
9,maryland,governor,martin o'malley,d,2007-01-17,2015


In [25]:
section = pd.read_html(archive[18])[4]
section = section.iloc[1:len(section),:]
section.columns = ["State", "Chamber", "Name", "Party", "Date", "End", "DeleteC"]
del(section["DeleteC"])
section["Chamber"] = ["governor" for x in range(len(section))]
section["State"] = section["State"].str.lower()
section["Party"] = section["Party"].str.lower()
section["Name"] = section["Name"].str.lower()
section["End"] = [re.findall("\d\d\d\d",x)[0] for x in section["End"]]
dates = section["Date"].tolist()
year = [re.findall("\d\d\d\d",x)[0] for x in dates]
day = [re.sub(",","",re.findall("\d+,",x)[0]) for x in dates]
month = [mdict[re.findall("^[A-Z][a-z]+",x)[0]] for x in dates]
df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
date = pd.to_datetime(df)
section = section.reset_index(drop = True)
section["Date"] = date
section["Party"] = [x[0] for x in section["Party"]]
section = section.sort_values(by = "Date").reset_index(drop=True)
weird = section["Name"].str.replace(",","")
#section["Name"] = pd.Series([x[int(len(x)/2):len(x)] for x in weird])
section18 = section.copy(deep=True)
section18

,State,Chamber,Name,Party,Date,End
0,texas,governor,rick perry,r,2000-12-21,2015
1,mississippi,governor,haley barbour,r,2004-01-13,2012
2,montana,governor,brian schweitzer,d,2005-01-03,2013
3,new hampshire,governor,john lynch,d,2005-01-06,2013
4,indiana,governor,mitch daniels,r,2005-01-10,2013
5,washington,governor,christine gregoire,d,2005-01-12,2013
6,nebraska,governor,dave heineman,r,2005-01-20,2015
7,idaho,governor,butch otter,r,2007-01-01,2015
8,massachusetts,governor,deval patrick,d,2007-01-04,2015
9,arkansas,governor,mike beebe,d,2007-01-09,2015


In [26]:
section = pd.read_html(archive[19])[4]
section = section.iloc[1:len(section),:]
section.columns = ["State", "Chamber", "Name", "Party", "Date", "End", "DeleteC"]
del(section["DeleteC"])
section["Chamber"] = ["governor" for x in range(len(section))]
section["State"] = section["State"].str.lower()
section["Party"] = section["Party"].str.lower()
section["Name"] = section["Name"].str.lower()
section["End"] = [re.findall("\d\d\d\d",x)[0] for x in section["End"]]
dates = section["Date"].tolist()
year = [re.findall("\d\d\d\d",x)[0] for x in dates]
day = [re.sub(",","",re.findall("\d+,",x)[0]) for x in dates]
month = [mdict[re.findall("^[A-Z][a-z]+",x)[0]] for x in dates]
df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
date = pd.to_datetime(df)
section = section.reset_index(drop = True)
section["Date"] = date
section["Party"] = [x[0] for x in section["Party"]]
section = section.sort_values(by = "Date").reset_index(drop=True)
weird = section["Name"].str.replace(",","")
#section["Name"] = pd.Series([x[int(len(x)/2):len(x)] for x in weird])
section19 = section.copy(deep=True)
section19

,State,Chamber,Name,Party,Date,End
0,texas,governor,rick perry,r,2000-12-21,2015
1,south dakota,governor,mike rounds,r,2003-01-07,2011
2,maine,governor,john baldacci,d,2003-01-08,2011
3,vermont,governor,jim douglas,r,2003-01-09,2011
4,oklahoma,governor,brad henry,d,2003-01-13,2011
5,georgia,governor,sonny perdue,r,2003-01-13,2011
6,oregon,governor,ted kulongoski,d,2003-01-13,2011
7,south carolina,governor,mark sanford,r,2003-01-15,2011
8,tennessee,governor,phil bredesen,d,2003-01-18,2011
9,alabama,governor,bob riley,r,2003-01-20,2011


In [27]:
section = pd.read_html(archive[20])[5]
section = section.iloc[1:len(section),:]
section.columns = ["State", "Chamber", "Name", "Party", "Date", "End", "DeleteC"]
del(section["DeleteC"])
section["Chamber"] = ["governor" for x in range(len(section))]
section["State"] = section["State"].str.lower()
section["Party"] = section["Party"].str.lower()
section["Name"] = section["Name"].str.lower()
section["End"] = [re.findall("\d\d\d\d",x)[0] for x in section["End"]]
dates = section["Date"].tolist()
year = [re.findall("\d\d\d\d",x)[0] for x in dates]
day = [re.sub(",","",re.findall("\d+,",x)[0]) for x in dates]
month = [mdict[re.findall("^[A-Z][a-z]+",x)[0]] for x in dates]
df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
date = pd.to_datetime(df)
section = section.reset_index(drop = True)
section["Date"] = date
section["Party"] = [x[0] for x in section["Party"]]
section = section.sort_values(by = "Date").reset_index(drop=True)
weird = section["Name"].str.replace(",","")
#section["Name"] = pd.Series([x[int(len(x)/2):len(x)] for x in weird])
section20 = section.copy(deep=True)
section20

,State,Chamber,Name,Party,Date,End
0,north dakota,governor,john hoeven,r,2000-12-15,2012
1,texas,governor,rick perry,r,2000-12-21,2014
2,hawaii,governor,linda lingle,r,2002-12-02,2010
3,new mexico,governor,bill richardson,d,2003-01-01,2010
4,michigan,governor,jennifer granholm,d,2003-01-01,2010
5,wyoming,governor,dave freudenthal,d,2003-01-06,2010
6,minnesota,governor,tim pawlenty,r,2003-01-06,2010
7,wisconsin,governor,jim doyle,d,2003-01-06,2010
8,south dakota,governor,mike rounds,r,2003-01-07,2010
9,rhode island,governor,donald carcieri,r,2003-01-07,2010


In [28]:
govs = pd.concat([section, section0,\
                 section1, section2,\
                 section3, section4,\
                 section5, section6,\
                 section7, section8,\
                 section9, section10,\
                 section11, section12,\
                 section13, section14,\
                 section15, section16,\
                 section17, section18,\
                 section19, section20]).drop_duplicates()
govs

,State,Chamber,Name,Party,Date,End
0,north dakota,governor,john hoeven,r,2000-12-15,2012
1,texas,governor,rick perry,r,2000-12-21,2014
2,hawaii,governor,linda lingle,r,2002-12-02,2010
3,new mexico,governor,bill richardson,d,2003-01-01,2010
4,michigan,governor,jennifer granholm,d,2003-01-01,2010
5,wyoming,governor,dave freudenthal,d,2003-01-06,2010
6,minnesota,governor,tim pawlenty,r,2003-01-06,2010
7,wisconsin,governor,jim doyle,d,2003-01-06,2010
8,south dakota,governor,mike rounds,r,2003-01-07,2010
9,rhode island,governor,donald carcieri,r,2003-01-07,2010


# Final Preprocessing before shipping of product

In [29]:
govs = pd.concat([section, section0,\
                 section1, section2,\
                 section3, section4,\
                 section5, section6,\
                 section7, section8,\
                 section9, section10,\
                 section11, section12,\
                 section13, section14,\
                 section15, section16,\
                 section17, section18,\
                 section19, section20]).drop_duplicates()
st_abr = {k.lower():v for k,v in STATES_DICT.items()} #created a dictionary that goes from lowercase states to abbreviations
govs["Name"] = govs["Name"].str.replace(r"[^a-z ]","") #removes all non letters and non spaces in names
govs["State"] = govs['State'].map(st_abr) #turn states into abbreviations
govs.columns = ["state", "chamber", "name", "party", "date", "end"] #this line I make column names lowercase
govs = govs[["date", "party", "name", "chamber", "state", "end"]] #this line I reorder columns
govs = govs.sort_values(by="date").reset_index(drop = True) #this line I sort by date
govs
#date, party, name, chamber, state

,date,party,name,chamber,state,end
0,2000-12-15,r,john hoeven,governor,ND,2012
1,2000-12-21,r,rick perry,governor,TX,2015
2,2000-12-21,r,rick perry,governor,TX,2014
3,2002-12-02,r,linda lingle,governor,HI,2010
4,2003-01-01,d,bill richardson,governor,NM,2010
5,2003-01-01,d,jennifer granholm,governor,MI,2010
6,2003-01-06,d,dave freudenthal,governor,WY,2010
7,2003-01-06,r,tim pawlenty,governor,MN,2010
8,2003-01-06,d,jim doyle,governor,WI,2010
9,2003-01-07,r,mike rounds,governor,SD,2010


In [30]:
#govs.to_csv("Clean_Governor_Winners.csv")

In [33]:
govs.drop(index=187).reset_index(drop = True)

,date,party,name,chamber,state,end
0,2000-12-15,r,john hoeven,governor,ND,2012
1,2000-12-21,r,rick perry,governor,TX,2015
2,2000-12-21,r,rick perry,governor,TX,2014
3,2002-12-02,r,linda lingle,governor,HI,2010
4,2003-01-01,d,bill richardson,governor,NM,2010
5,2003-01-01,d,jennifer granholm,governor,MI,2010
6,2003-01-06,d,dave freudenthal,governor,WY,2010
7,2003-01-06,r,tim pawlenty,governor,MN,2010
8,2003-01-06,d,jim doyle,governor,WI,2010
9,2003-01-07,r,mike rounds,governor,SD,2010


In [34]:
#govs.to_csv("Clean_Governor_Winners.csv")